### Let's build a neuron and neuron network

This is the activation Functions

In [4]:
fun sigmoid(x: Double): Double = 1.0 / (1.0 + exp(-x))
fun sigmoidDerivative(x: Double): Double = x * (1 - x)

And the neuron definition

In [5]:
import kotlin.random.Random

data class Neuron(var weights: DoubleArray, var bias: Double = Random.nextDouble())


In [6]:
class NeuralNetwork(
    val inputSize: Int,
    val hiddenSize: Int,
    val outputSize: Int,
    val learningRate: Double = 0.1
) {

    private val hiddenLayer = Array(hiddenSize) { Neuron(DoubleArray(inputSize) { Random.nextDouble() }) }
    private val outputLayer = Array(outputSize) { Neuron(DoubleArray(hiddenSize) { Random.nextDouble() }) }

    fun forward(input: DoubleArray): DoubleArray {
        val hiddenOutputs = hiddenLayer.map { neuron ->
            val sum = neuron.weights.zip(input).sumOf { it.first * it.second } + neuron.bias
            sigmoid(sum)
        }.toDoubleArray()

        return outputLayer.map { neuron ->
            val sum = neuron.weights.zip(hiddenOutputs).sumOf { it.first * it.second } + neuron.bias
            sigmoid(sum)
        }.toDoubleArray()
    }

    fun train(inputs: List<DoubleArray>, targets: List<DoubleArray>, epochs: Int = 10000) {
        repeat(epochs) {
            for ((input, target) in inputs.zip(targets)) {

                // --- FORWARD PASS ---
                val hiddenOutputs = hiddenLayer.map { neuron ->
                    val z = neuron.weights.zip(input).sumOf { (w, x) -> w * x } + neuron.bias
                    sigmoid(z)
                }.toDoubleArray()

                val finalOutputs = outputLayer.map { neuron ->
                    val z = neuron.weights.zip(hiddenOutputs).sumOf { (w, x) -> w * x } + neuron.bias
                    sigmoid(z)
                }.toDoubleArray()

                // --- BACKWARD PASS ---

                // 1. Output layer
                val outputErrors = target.zip(finalOutputs).map { (t, o) -> t - o }.toDoubleArray()
                val outputGradients = finalOutputs.mapIndexed { i, o ->
                    outputErrors[i] * sigmoidDerivative(o)
                }

                outputLayer.forEachIndexed { i, neuron ->
                    neuron.weights = neuron.weights.mapIndexed { j, w ->
                        w + learningRate * outputGradients[i] * hiddenOutputs[j]
                    }.toDoubleArray()
                    neuron.bias += learningRate * outputGradients[i]
                }

                // 2. Hidden layer
                val hiddenErrors = hiddenLayer.indices.map { h ->
                    outputLayer.indices.sumOf { o ->
                        outputGradients[o] * outputLayer[o].weights[h]
                    }
                }

                val hiddenGradients = hiddenOutputs.mapIndexed { i, hOut ->
                    hiddenErrors[i] * sigmoidDerivative(hOut)
                }

                hiddenLayer.forEachIndexed { i, neuron ->
                    neuron.weights = neuron.weights.mapIndexed { j, w ->
                        w + learningRate * hiddenGradients[i] * input[j]
                    }.toDoubleArray()
                    neuron.bias += learningRate * hiddenGradients[i]
                }
            }
        }
    }
}

---
### Example usage for (XOR)

In [7]:
val neuralNetwork = NeuralNetwork(inputSize = 2, hiddenSize = 2, outputSize = 1)

val inputs = listOf(
    doubleArrayOf(0.0, 0.0),
    doubleArrayOf(0.0, 1.0),
    doubleArrayOf(1.0, 0.0),
    doubleArrayOf(1.0, 1.0),
)

val targets = listOf(
    doubleArrayOf(0.0),
    doubleArrayOf(1.0),
    doubleArrayOf(1.0),
    doubleArrayOf(0.0),
)

neuralNetwork.train(inputs, targets, epochs = 100000)

buildString {
    append("Testing XOR after traning:\n")
    for (input in inputs) {
        val output = neuralNetwork.forward(input)
        append("${input.joinToString()} -> ${"%.4f".format(output[0])}\n")
    }
}

Testing XOR after traning:
0.0, 0.0 -> 0.0131
0.0, 1.0 -> 0.9888
1.0, 0.0 -> 0.9888
1.0, 1.0 -> 0.0115
